In [ ]:
!sudo apt update
!sudo apt install -y tesseract-ocr
!pip install pytesseract pillow python-docx langchain langchain-community faiss-cpu sentence-transformers

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,729 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:13 https://r2u.stat.illi

In [ ]:
from google.colab import files
uploaded = files.upload()  # drag and drop your HR.docx here
docx_path = next(iter(uploaded))  # gets the filename

Saving iLoveMerge.docx to iLoveMerge.docx


In [ ]:
!pip install docx2txt

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader(docx_path)
docs_text = loader.load()
doc_text = "\n".join([doc.page_content for doc in docs_text])


#from langchain_community.document_loaders import TextLoader

#loader = TextLoader("iLoveMerge.txt", encoding="utf-8")  # change path and encoding if needed
#docs_text = loader.load()
#doc_text = "\n".join([doc.page_content for doc in docs_text])

In [ ]:
doc_text


'--- Page 1 ---\n\nInsurance Regulatory and Development Authority (Meetings) Regulations, 20001 \n\n \n\n(Amended up to 21.10.2013) \n\n \n\nF.No.IRDA/Reg./7/2000. In exercise of the powers conferred by sub-section (4) of \n\nSection 10 read with clauses (a) and (b) of sub-section (2) of section 26 of the \n\nInsurance Regulatory and Development Authority Act, 1999 (41 of 1999), the \n\nAuthority, in consultation with the Insurance Advisory Committee, hereby makes the \n\nfollowing regulations, namely: \n\n \n\n1. \n\nShort title, extent and commencement: \n\n \n\n(1) \n\nThese regulations may be called the Insurance Regulatory and Development \n\nAuthority (Meetings) Regulations, 2000. \n\n(2) \n\nThey shall come into force on the date of their publication in the Official \n\nGazette. \n\n \n\n2. \n\nDefinitions: \n\nIn these regulations, unless the context otherwise requires, \n\n \n\n(a) \n\nAct means the Insurance Regulatory and Development Authority Act, \n\n1999(41 of 1999). \n\n

In [ ]:
from docx import Document

In [ ]:
def extract_tables(docx_path):
    doc = Document(docx_path)
    table_data = []
    for table in doc.tables:
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            if any(row_data):
                table_data.append(" | ".join(row_data))
    return "\n".join(table_data)

tables_text = extract_tables(docx_path)


In [ ]:
tables_text

''

In [ ]:
#OCR image content from .docx using Tesseract
from PIL import Image
from IPython.display import display
from zipfile import ZipFile
from io import BytesIO
import pytesseract

In [ ]:
def extract_images_and_ocr(docx_path, lang='eng'):
    image_texts = []
    success_count = 0
    try:
        with ZipFile(docx_path) as z:
            for file in z.namelist():
                if file.startswith("word/media/") and file.lower().endswith((".png", ".jpg", ".jpeg")):
                    print(f"🔍 Processing image: {file}")
                    image_bytes = z.read(file)
                    image = Image.open(BytesIO(image_bytes))

                    # Run OCR
                    text = pytesseract.image_to_string(image, lang=lang)

                    if text.strip():
                        print(f"✅ OCR text extracted from {file}")
                        image_texts.append(f"[{file}]\n{text.strip()}")
                        success_count += 1
                    else:
                        print(f"⚠️ No text found in {file} — displaying image:")
                        display(image)  # Show image in Colab for visual check

        print(f"\n📊 Total images processed: {success_count} with text extracted")
        return "\n\n".join(image_texts), success_count


    except Exception as e:
        print("❌ Error extracting images or running OCR:", e)
        return ""

In [ ]:
ocr_text, ocr_count = extract_images_and_ocr(docx_path)

# Then check if any text was extracted
if ocr_text.strip():
    print("\n📄 Final OCR Text Output:\n")
    print(ocr_text)
else:
    print("❌ No text was extracted from any image.")

print(f"\n📊 Total successful OCRs: {ocr_count}")



📊 Total images processed: 0 with text extracted
❌ No text was extracted from any image.

📊 Total successful OCRs: 0


In [ ]:
full_text = f"{doc_text}\n\n[TABLE DATA]\n{tables_text}\n\n[IMAGE OCR TEXT]\n{ocr_text}"

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.create_documents([full_text])

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-26-dc99f1f23635>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunks, embedding_model)
print("✅ FAISS vector store created")

✅ FAISS vector store created


In [ ]:
query = input(" Ask something from your HR document: ")
results = db.similarity_search(query, k=1)

 Ask something from your HR document: what happens when quorum is not present


In [ ]:
print("\n📄 Top Matching Chunks:\n")
for i, res in enumerate(results, 1):
    print(f"{i}. {res.page_content}\n")


📄 Top Matching Chunks:

1. --- Page 3 ---

(3) 

If at any meeting, quorum is not present, the Chairperson or the presiding 

member, as the case may be, shall after waiting for thirty minutes, adjourn the 

meeting for such hour on some other day as he may think fit and the notice of 

such adjournment shall be given to all the members and the business which was 

to have been brought before the original meeting, had there been a quorum,

